## Item Clipping and Download

This example clips, downloads and unzips a given scene to the AOI provided, returning only the image data contained in with the AOI. To so this, we use the ["Clips" API](https://www.planet.com/docs/reference/clips-api/).

In this example, we are clipping a scene to contain only Golden Gate Park in San Francisco.

### Input Parameters 

In [ ]:
from os import environ

# Set API key (this should to be an environment variable)
api_key = environ['PL_API_KEY']

# Sent Scene ID
scene_id = '20170517_192201_1056516_RapidEye-2'

# Set Item Type
item_type = 'REOrthoTile'

# Set Asset Type
asset_type = 'visual'

# Area Of Interest (clip geometry) in GeoJSON format
aoi_json = '''{
        "type": "Polygon",
        "coordinates": [
          [
            [
              -122.51103401184083,
              37.771596736802074
            ],
            [
              -122.51060485839844,
              37.763997637045456
            ],
            [
              -122.45902061462401,
              37.76603318676243
            ],
            [
              -122.45773315429689,
              37.7654903789825
            ],
            [
              -122.45275497436523,
              37.76637243960179
            ],
            [
              -122.45455741882324,
              37.775124624817906
            ],
            [
              -122.46597290039062,
              37.7738356083287
            ],
            [
              -122.51103401184083,
              37.771596736802074
            ]
          ]
        ]
      }'''

### Clip scenes to AOI, poll status, and download when complete

In [ ]:
import json
import requests
import time

# Construct clip API payload
clip_payload = {
    'aoi': json.loads(aoi_json),
    'targets': [
      {
        'item_id': scene_id,
        'item_type': item_type,
        'asset_type': asset_type
      }
    ]
}

# Request clip of scene (This will take some time to complete)
request = requests.post('https://api.planet.com/compute/ops/clips/v1', auth=(api_key, ''), json=clip_payload)
clip_url = request.json()['_links']['_self']

# Poll API to monitor clip status. Once finished, download and upzip the scene
clip_succeeded = False
while not clip_succeeded:

    # Poll API
    check_state_request = requests.get(clip_url, auth=(api_key, ''))
    
    # If clipping process succeeded , we are done
    if check_state_request.json()['state'] == 'succeeded':
        clip_download_url = check_state_request.json()['_links']['results'][0]
        clip_succeeded = True
        print("Clip of scene succeeded and is ready to download") 
    
    # Still activating. Wait 1 second and check again.
    else:
        print("...Still waiting for clipping to complete...")
        time.sleep(1)

### Download and upzip the clip
Once complete, look in the output directory to see your clipped tif file.

NOTE: Clipped scene will only be available for 5 minutes.

In [ ]:
import os
from tqdm import tqdm
import zipfile

# Download clip
response = requests.get(clip_download_url, stream=True)
with open('output/' + scene_id + '.zip', "wb") as handle:
    for data in tqdm(response.iter_content()):
        handle.write(data)

# Unzip file
ziped_item = zipfile.ZipFile('output/' + scene_id + '.zip')
ziped_item.extractall('output/' + scene_id)    
  
# Delete zip file
os.remove('output/' + scene_id + '.zip')
print('Downloaded clips located in: output/')